In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
from pandas import DataFrame

In [2]:
def max_stock(data):
    L = []
    for i in set(data["stock"]):
        counter = list(data["stock"].values).count(i)
        L.append(counter)                                        ###L为各股票出现的次数，即交易天数
    stock_id = list(set(data["stock"]))
    stock_m = stock_id[L.index(max(L))]                           ###选出交易天数最全的一只股票
    return stock_m

In [3]:
def best_perf(data, T, N, K):                                 ###data为原始数据，T为所要比较的涨跌幅的日期，N为T之前的时间跨度，K为最大成交总额及涨跌幅对比数
    data = data[['date','stock','close','turnover_sum','change']]
    data['stock'] = data['stock'].astype(str)              ###更改stock的数据类型为字符型
    
    stock_m = max_stock(data)
    stock_eg = data[data['stock'] == stock_m]

    start = int(stock_eg[stock_eg['date'] ==  T].index.values-N-1)-stock_eg.index[0]      ###寻找T前N+1天的索引位置
    end = int(stock_eg[stock_eg['date'] ==  T].index.values)-stock_eg.index[0]             ###寻找T日的索引位置
    slc = stock_eg.iloc[[start,end-1,end],:]
    
    init = data[data['date'] == slc['date'].values[0]]
    fina = data[data['date'] == slc['date'].values[1]]
    fina1 = data[data['date'] == slc['date'].values[2]]
    secl_sum = init.append(fina).sort_index()
    secl_sum = secl_sum.append(fina1).sort_index()                    ####将所有在T,T-N-1,T-1时有数据的股票信息合成一张表
    
    stock_intersect = list(set(init['stock']).intersection(set(fina['stock'])))
    stock_intersect = list(set(stock_intersect).intersection(set(fina1['stock'])))    ###选出在上述三个时段都有数据的股票
    
    starter = secl_sum[secl_sum['stock'] == stock_intersect[0]]
    turn_o = starter['turnover_sum']
    differ = turn_o.values[1]-turn_o.values[0]
    stock_f = starter[starter['date'] == T]
    stock_f.insert(5,'turnover_N',[differ])                          ###将上述选出的股票计算N天的成交量总额及T日的涨跌幅
    
    for k in stock_intersect[1:]:
        a = secl_sum[secl_sum['stock'] == k]
        turn_o = a['turnover_sum']
        differ = turn_o.values[1]-turn_o.values[0]
        b = a[a['date'] == T]
        b.insert(5,'turnover_N',[differ])
        stock_f = stock_f.append(b)
    stock_f = stock_f.sort_index()                                 ###循环进行上述操作
    
    stock_Tmax = stock_f.sort_values(by = ['turnover_N'], ascending = False).head(K)     ###取出N天成交金额最大的K个股票
    stock_Cmax = stock_f.sort_values(by = ['change'], ascending = False).head(K)         ###取出T日涨跌幅最大的K个股票
    stock_best = list(set(stock_Tmax['stock']).intersection(set(stock_Cmax['stock'])))   ###取出涨跌幅及成交金额都是最大K的股票     
    
    stock_bestinf = stock_Cmax[stock_Cmax['stock'] == stock_best[0]]
    for i in stock_best[1:]:
        infom = stock_Cmax[stock_Cmax['stock'] == i]
        stock_bestinf = stock_bestinf.append(infom)                                ###获取这几只股票的具体信息
        stock_bestinf = stock_bestinf.sort_index()
    return stock_bestinf    
    
    
    
    

In [4]:
data = pd.read_csv('D:/Documents/Go Abroad/实习/成交金额/Python数据/2017.csv',encoding='gbk',engine='python')

In [5]:
stock_bestinf = best_perf(data, '2017-12-28', 20, 100)

D:\Anaconda3\envs\dayu\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [8]:
stock_bestinf = stock_bestinf.sort_index()
stock_bestinf

,date,stock,close,turnover_sum,change,turnover_N
75328,2017-12-28,717,8.85,9.554107e+10,0.047337,1.692820e+10
89370,2017-12-28,807,10.35,1.673262e+11,0.094080,1.435638e+10
134662,2017-12-28,2027,14.04,1.194158e+11,0.043866,1.508705e+10
388719,2017-12-28,601899,4.69,1.312115e+11,0.100939,1.419104e+10
431562,2017-12-28,600111,14.81,2.834909e+11,0.066235,1.534416e+10
644933,2017-12-28,603993,7.01,2.586286e+11,0.068598,1.710043e+10
679840,2017-12-28,600519,718.69,4.543575e+11,0.082071,5.897544e+10
